### 概率算法定义


In [2]:
from fractions import Fraction
import itertools

In [3]:
# 基础概率计算
def step(s, i):
    '''强化到第i个位的概率。s是长度4的数组，0表示空栏。若i位置为0返回出新属性的总概率。'''
    count = s.count(0)
    if count == 0:
        return Fraction(s[i], sum(s))
    return Fraction(s[i] if s[i] > 0 else 7 + count, sum(s) - (4 - count) + 11)
def step2(s, i):
    '''强化到第i个位的概率。s是长度4以内的数组，不含0。若i超过s长度则返回新增某一个新属性的概率。'''
    if len(s) == 4:
        return Fraction(s[i], sum(s))
    return Fraction(s[i] if i < len(s) else 1, sum(s) - len(s) + 11)

In [4]:
def to(s, d):
    '''从s强化到d的概率。参数是长度4的数组。
    '''
    if s == d:
        return Fraction(1)
    result = 0
    end = 4 - s.count(0) + 1
    if end > 4:
        end = 4
    for i in range(end):
        if d[i] > s[i]:
            p = step(s, i)
            s[i] += 1
            result += p * to(s, d)
            s[i] -= 1
    return result

In [5]:
to([1,1,1,1],[1,1,1,5])

Fraction(1, 35)

In [6]:
to([1,1,1,0],[1,1,5,1])

Fraction(3368, 105105)

In [7]:
def possible(s, d, t):
    '''从s开始强化最多t次后，能满足d的概率。s和d是字典。'''
    # 考虑未出现属性是否有位置出现
    count = 0
    sk = s.keys()
    for key in d.keys():
        if key not in sk:
            count += 1
    if count > 4 - len(sk):
        return False
    # 考虑份数是否足够
    count = 0
    for key in d.keys():
        if key not in sk:
            count += d[key]
        elif s[key] < d[key]:
            count += d[key] - s[key]
        if count > t:
            return False
    return True

In [8]:
def satisfy(s, d):
    '''s是否已经满足d。s和d是字典。'''
    for k, v0 in d.items():
        v = s.get(k)
        if v == None or v < v0:
            return False
    return True

In [9]:
def choose(n, c):
    '''组合数，n个里面选c个'''
    result = 1
    for i in range(n, n - c, -1):
        result *= i
    for i in range(2, c + 1):
        result /= i
    return result

In [10]:
_e6 = 15750 # 6星强化到3所需经验
_e = _e6
# 副属性等级：到下一次强化副属性所需的御魂经验
exp = [_e, _e*2, _e*3, _e*4, _e*5]
print exp

[15750, 31500, 47250, 63000, 78750]


In [11]:
def least(s, d, t=5, l=0):
    '''从s开始强化t次后，满足d的概率。l是初始强化等级，无强化则0。s和d是字典。
    返回（成功的概率，平均消耗经验）'''
    if satisfy(s, d):
        return (Fraction(1), 0)
    if t == 0 or not possible(s, d, t):
        return (Fraction(0), 0)
    result = 0
    exper = 0 # 得到结果（成功或失败）的期望消耗
    sk = s.keys()
    sv = s.values()
    for i, k in enumerate(sk): # 遍历强化到第i个属性
        p = step2(sv, i)
        s[k] += 1
        temp = least(s, d, t-1, l+1)
        result += p * temp[0]
        exper += p * (exp[l] + temp[1])
        s[k] -= 1
    if len(sv) < 4: # 额外考虑出新属性的情况
        p = step2(sv, len(sv))
        for newkey in range(11):
            if newkey not in sk:
                s2 = dict(s)
                s2[newkey] = 1
                temp = least(s2, d, t-1, l+1)
                result += p * temp[0]
                exper += p * (exp[l] + temp[1])
    return result, exper

In [12]:
least({1:1,2:1,3:1},{1:1,2:1,3:1,4:5})

(Fraction(1, 385), Fraction(226800, 11))

In [13]:
def random(d, n=4):
    '''随机的初始n个属性的御魂能强化后能满足d的概率。d是字典'''
    result = 0
    exper = 0
    dk = d.keys()
    l = len(dk)
    others = []
    for key in range(11):
        if key not in dk:
            others.append(key)
    for i in range(min(5-n, l+1)): # 初始属性中没有i个期望属性
        i0 = l - i # 初始属性中有i0个期望属性
        p = Fraction(choose(l, i0) * choose(11-i0, n-i0), choose(11, n))
        for keys in itertools.combinations(dk, i0): # 组合i0个属性
            s = {}
            for key in keys:
                s[key] = 1
            for i in range(len(keys), n):
                s[others[i]] = 1
            temp = least(s, d)
            result += p * temp[0]
            exper += p * temp[1]
    return result, exper

In [14]:
class Set():
    def __init__(self, *args):
        self.all = args
    def satisfy(self, s):
        for d in self.all:
            if satisfy(s, d):
                return True
        return False
    def possible(self, s, t):
        for d in self.all:
            if possible(s, d, t):
                return True
        return False

In [15]:
def least2(s, d, t=5, l=0):
    '''从s开始强化t次后，满足d的概率。l是初始强化等级，无强化则0。s是字典，d是Set
    返回（成功的概率，平均消耗经验）'''
    if d.satisfy(s):
        return (Fraction(1), 0)
    if t == 0 or not d.possible(s, t):
        return (Fraction(0), 0)
    result = 0
    exper = 0 # 得到结果（成功或失败）的期望消耗
    sk = s.keys()
    sv = s.values()
    for i, k in enumerate(sk): # 遍历强化到第i个属性
        p = step2(sv, i)
        s[k] += 1
        temp = least2(s, d, t-1, l+1)
        result += p * temp[0]
        exper += p * (exp[l] + temp[1])
        s[k] -= 1
    if len(sv) < 4: # 额外考虑出新属性的情况
        p = step2(sv, len(sv))
        for newkey in range(11):
            if newkey not in sk:
                s2 = dict(s)
                s2[newkey] = 1
                temp = least2(s2, d, t-1, l+1)
                result += p * temp[0]
                exper += p * (exp[l] + temp[1])
    return result, exper

In [ ]:
def buy(d, n, t=5):
    '''初始属性n个，强化t次满足d的概率。'''
    dn = 

In [16]:
def buy2(d, t=5):
    '''赌御魂，强化t次满足d的概率。'''
    return buy(d, 2) * 1.0/2 + 
            buy(d, 3) * 1.0/3 + 
            buy(d, 4) * 1.0/6;

In [36]:
d={1:1,2:1,3:1}

In [37]:
d.keys()

[1, 2, 3]

In [20]:
least2({1:1,2:1,3:1}, Set({1:5}))

(Fraction(3473, 105105), Fraction(945225, 13))

In [34]:
least2({1:1,2:1}, Set({1:4,3:1}))

(Fraction(743269, 48096048), Fraction(5910758625, 81796))

In [35]:
least2({1:1,2:1}, Set({1:1,3:4}))

(Fraction(1037, 120120), Fraction(187193125, 3146))

'''提供手游阴阳师中御魂强化的一些概率计算，计算方式由本人经验得出（后在nga上也有其他人得到相同结论）。
定义：御魂强化前，定义每个副属性都有**1份**，每次强化其中某一个属性，则那个属性**份数加1**。
假定每一次强化都是一次独立的概率计算，概率只和强化前各属性**份数**相关：
* 副属性满4条时，下一次**强化在x属性的概率=x的份数/总份数**
* 副属性不满4条时，额外定义**未出现的属性份数为1**，强化将在所有11种属性中随机（即可能出现新属性）

1. 对一个已知御魂，强化到一个或多个期望状态的概率。
2. 从初始随机n个属性的御魂强化到一个状态的概率。
典型用例：
1. to([1,1,1,0], [6,1,1,0])
初始3属性，全部强化到第一个属性的概率
2. least({1:1,2:1,3:1}, {1:4})
初始3属性，强化第一个属性3次的概率，和平均每个御魂消耗的经验（按6星算）
比如碰到3属性带暴击，这就是11爆的概率（默认最多强满5次）
3. random({1:4}, n)
初始随机n属性，强化指定一个属性3次的概率，和平均每个御魂消耗的经验（按6星算）
另外结合6星初始属性数的概率分布，可以算出随机6星强化到特定属性3次的概率
注意这里是指定某一个属性（比如想要11爆），并不是说任何一个属性强化3次都算成功
4. least2({1:1,2:1,3:1,4:1}, Set({1:5,2:2},{1:5,3:2},{1:5,4:2}))
初始4属性，强化第一个属性4次，强化另外3个属性1次的概率

参数含义解释：
数组：数字代表份数
字典：键是0到10代表11种属性（没有语义对应关系），值代表对应的份数。
数组长度和字典键数都不能超过4。'''